In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import scienceplots

import time
import math
import os
import random
from functools import partial
from decimal import Decimal
import numpy as np
import scipy.io as sio
import pysindy as ps
from tqdm import trange

from pymoo_ga import *
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.dnsga2 import DNSGA2
from pymoo.termination.default import DefaultMultiObjectiveTermination
from pymoo.optimize import minimize
from pymoo.core.problem import StarmapParallelization
from multiprocessing.pool import ThreadPool

from utils import *
from skimage.restoration import estimate_sigma
import bm3d
# from okridge.solvel0 import *
from solvel0 import solvel0, MIOSR
from best_subset import backward_refinement, brute_force_all_subsets
from UBIC import *
from kneed import KneeLocator
from bayesian_model_evidence import log_evidence

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.preprocessing import StandardScaler
from sklearn import covariance
from sklearn.metrics import mean_absolute_percentage_error

mrmr is not installed in the env you are using. This may cause an error in future if you try to use the (missing) lib.
L0BnB is not installed.


In [2]:
n_poly = 6
n_derivatives = 6
n_modules = 8

In [3]:
data_path = "../PDE-Discovery-EC/Datasets/"
data = sio.loadmat(os.path.join(data_path, "burgers.mat"))
u_clean = (data['usol']).real; u = u_clean.copy()
x = (data['x'][0]).real
t = (data['t'][:,0]).real
dt = t[1]-t[0]; dx = x[2]-x[1]

### Add noise

In [4]:
np.random.seed(0)
noise_type = "gaussian"
noise_lv = float(50)
print("Noise level:", noise_lv)
noise = 0.01*np.abs(noise_lv)*(u.std())*np.random.randn(u.shape[0],u.shape[1])
u = u + noise

Noise level: 50.0


### Denoise

In [5]:
# bm3d_file = f"./Denoised_data/burgers_{noise_type}{int(noise_lv)}_bm3d.npy"
# load_denoised_data = True
# if load_denoised_data:
#     print("Loading denoised data...")
#     u = np.load(bm3d_file)
# else:
#     kernel = RBF(length_scale=1, length_scale_bounds=(1e-2, 1e3)) + \
#                     WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e10))
    
#     xx = colvec(x)
#     u_mean = np.copy(u)
#     u_std = np.ones(u.shape)
#     for i in trange(len(t)):    
#         gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.0, 
#                                        n_restarts_optimizer=10 # 20
#                                       )
    
#         gpr.fit(xx, u_mean[:, i])
#         um, ustd = gpr.predict(xx, return_std=True)
#         u_mean[:, i] = um
#         u_std[:, i] = ustd
    
#     est_sigma = u_std.mean() # max also works well
#     # est_sigma = (est_sigma+estimate_sigma(u))/2
#     u = bm3d.bm3d(u, sigma_psd=est_sigma, 
#                   stage_arg=bm3d.BM3DStages.ALL_STAGES, 
#                   blockmatches=(False, False))
    
#     ### to save ###
#     # np.save(bm3d_file, u)

np.random.seed(0)
fake_noise = np.random.normal(loc=0.0, scale=estimate_sigma(u), size=u.shape)
sigmas = estimate_sigma(u+fake_noise)*np.arange(0.1, 2., 0.1)
est_sigma = sigmas[np.argmin([((u-bm3d.bm3d(u+fake_noise, sigma_psd=sigma, stage_arg=bm3d.BM3DStages.ALL_STAGES, blockmatches=(False, False)))**2).mean() \
                              for sigma in sigmas])]
u = bm3d.bm3d(u, sigma_psd=est_sigma, 
                  stage_arg=bm3d.BM3DStages.ALL_STAGES, 
                  blockmatches=(False, False))

In [6]:
xt = np.array([x.reshape(-1, 1), t.reshape(1, -1)], dtype=object)
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T

In [7]:
function_library = ps.PolynomialLibrary(degree=n_poly, include_bias=False)

weak_lib = ps.WeakPDELibrary(
    function_library=function_library,
    derivative_order=n_derivatives,
    spatiotemporal_grid=XT,
    include_bias=True,
    diff_kwargs={"is_uniform":True},
    K=10000
)

X_pre = np.array(weak_lib.fit_transform(np.expand_dims(u, -1)))
y_pre = weak_lib.convert_u_dot_integral(np.expand_dims(u, -1))
feature_names = np.array(weak_lib.get_feature_names())

# R_path = "./Cache/"
# np.save(os.path.join(R_path, f"X_pre_burgers_noise{int(noise_lv)}.npy"), X_pre)
# np.save(os.path.join(R_path, f"y_pre_burgers_noise{int(noise_lv)}.npy"), y_pre)
# np.save(os.path.join(R_path, f"feature_names_burgers.npy"), feature_names)

In [8]:
base_poly = np.array([[p, 0] for p in range(1, n_poly+1)])
base_derivative = np.array([[0, d] for d in range(1, n_derivatives+1)])
modules = [(0, 0)] if weak_lib.include_bias else []
modules += [(p, 0) for p in range(1, n_poly+1)] + \
            [(0, d) for d in range(1, n_derivatives+1)] + \
            [tuple(p+d) for d in base_derivative for p in base_poly]
assert len(modules) == len(weak_lib.get_feature_names())
base_features = dict(zip(modules, X_pre.T))
u_t = y_pre.copy()

### Straightforward best-subset selection

In [9]:
# over 30 minutes for n_poly = 6 and n_derivatives = 6
# miosr_subsets = solvel0(X_pre, y_pre, miosr=True)

### Genetic algorithm with NSGA-II

In [10]:
pop_size = 500
order_complexity = False
pool = ThreadPool(4)
problem = PdeDiscoveryProblem(n_poly, n_derivatives, n_modules, 
                              base_features, u_t, order_complexity=order_complexity, 
                              elementwise_runner=StarmapParallelization(pool.starmap))

In [11]:
def add_prefix(file_path, prefix):
    dir_name, file_name = os.path.split(file_path)
    return os.path.join(dir_name, prefix + file_name)
    
load_pareto_front = False
n_max_gen = 200
n_max_evals = 100000
pf_file_path = f"./Cache/pf_SMSEMOA_burgers_noise{int(noise_lv)}.npy"

if not load_pareto_front:
    termination = DefaultMultiObjectiveTermination(
        xtol=1e-8,
        cvtol=1e-6,
        ftol=1e-8,
        period=50,
        n_max_gen=n_max_gen,
        n_max_evals=n_max_evals
    )
    
    from pymoo.algorithms.moo.sms import SMSEMOA
    from pymoo.algorithms.moo.age import AGEMOEA
    
    ### Optimization algorithms ###
    # algorithm = NSGA2(pop_size=pop_size,
    #                     sampling=PopulationSampling(),
    #                     crossover=GenomeCrossover(),
    #                     mutation=GenomeMutation(),
    #                     eliminate_duplicates=DuplicateElimination(),
    #                     )
    
    # algorithm = DNSGA2(pop_size=pop_size,
    #                 sampling=PopulationSampling(),
    #                 crossover=GenomeCrossover(),
    #                 mutation=GenomeMutation(),
    #                 eliminate_duplicates=DuplicateElimination(),
    #                 )
    
    algorithm = SMSEMOA(pop_size=pop_size,
                    sampling=PopulationSampling(),
                    crossover=GenomeCrossover(),
                    mutation=GenomeMutation(),
                    eliminate_duplicates=DuplicateElimination(),
                    )

    opt_time = time.time()
    res = minimize(problem, 
                   algorithm, 
                   termination=termination, 
                   verbose=True)
    opt_time = (time.time() - opt_time)/60
    print("Execution time:", opt_time)
    
    pareto_optimal_models = res.X
    # np.save(pf_file_path, pareto_optimal_models)

else:
    pareto_optimal_models = np.load(pf_file_path, allow_pickle=True)
    pass

np.random.seed(0)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      500 |      4 |             - |             -
     2 |     1000 |      5 |  0.0114103104 |         ideal
     3 |     1500 |      7 |  0.0536293805 |             f
     4 |     2000 |      6 |  0.0099541040 |         ideal
     5 |     2500 |      6 |  0.000000E+00 |             f
     6 |     3000 |      8 |  0.0158547574 |         ideal
     7 |     3500 |      7 |  0.0045456452 |             f
     8 |     4000 |      7 |  0.0060178784 |             f
     9 |     4500 |      8 |  0.0168372830 |             f
    10 |     5000 |      7 |  0.0029886969 |         ideal
    11 |     5500 |      7 |  0.000000E+00 |             f
    12 |     6000 |      7 |  0.0006141533 |             f
    13 |     6500 |      8 |  0.0041898584 |         ideal
    14 |     7000 |      9 |  0.0138895788 |             f
    15 |     7500 |      9 |  0.0000687940 |             f
    16 |     8000 |      9 |  0.0003291117 |            

In [12]:
### OPTIONAL: REFINE PARETO FRONT ###
from operator import itemgetter

effective_candidates = extract_unique_candidates(pareto_optimal_models)
new_pareto_optimal_models = []
for bs in backward_refinement([sorted([effective_candidates.index(_) for _ in list(pm[0])]) for pm in pareto_optimal_models], 
                              (problem.numericalize_genome(effective_candidates), y_pre)).get_best_subsets():
    bs = itemgetter(*bs)(effective_candidates)
    if type(bs[0]) is not tuple:
        bs = (bs,)
    new_pareto_optimal_models.append([frozenset(bs)])
pareto_optimal_models = np.array(new_pareto_optimal_models)
del new_pareto_optimal_models
pareto_optimal_models

array([[frozenset({(0, 1)})],
       [frozenset({(1, 1), (0, 2)})],
       [frozenset({(1, 1), (0, 2), (0, 4)})],
       [frozenset({(1, 1), (0, 2), (0, 4), (4, 3)})],
       [frozenset({(0, 4), (1, 1), (0, 2), (3, 6), (6, 3)})],
       [frozenset({(0, 4), (1, 1), (0, 6), (0, 2), (3, 6), (6, 3)})],
       [frozenset({(0, 4), (2, 1), (1, 1), (0, 6), (0, 2), (3, 6), (6, 3)})],
       [frozenset({(0, 4), (3, 4), (1, 1), (4, 6), (6, 6), (6, 4), (0, 2), (5, 3)})],
       [frozenset({(0, 4), (1, 1), (4, 2), (0, 6), (0, 2), (2, 2), (5, 3), (3, 2), (5, 2)})],
       [frozenset({(6, 2), (1, 2), (0, 4), (1, 1), (4, 2), (0, 6), (0, 2), (3, 6), (3, 2), (6, 3)})],
       [frozenset({(1, 2), (0, 4), (2, 1), (1, 1), (0, 6), (0, 2), (5, 6), (2, 2), (6, 6), (3, 2), (6, 3)})],
       [frozenset({(0, 4), (2, 1), (1, 1), (0, 3), (4, 2), (0, 6), (0, 2), (4, 5), (2, 2), (5, 3), (3, 2), (5, 2)})],
       [frozenset({(6, 2), (0, 4), (2, 1), (1, 1), (0, 3), (4, 6), (0, 6), (0, 2), (2, 2), (1, 0), (3, 2), (6, 3

### Top candidates by SHAP

In [13]:
max_ss = 16
feature_importance = dict(zip(effective_candidates, [0.0 for _ in range(len(effective_candidates))]))

for bs in pareto_optimal_models[1:]:
    bs = list(bs[0])
    shap_importance = shap_linear_importance(problem.numericalize_genome(bs), y_pre, scale=False)
    for i, _ in enumerate(bs):
        feature_importance[_] += shap_importance[i]

top_candidates = sorted([(v, k) for k, v in feature_importance.items()], reverse=True)
top_candidates = [v for k, v in top_candidates[:max_ss]]
feature_importance, top_candidates

({(0, 1): 0.0,
  (0, 2): 0.035402570542148584,
  (0, 3): 0.005285939068259219,
  (0, 4): 0.024273391779807526,
  (0, 5): 0.0018345718219225594,
  (0, 6): 0.00922654941647148,
  (1, 0): 0.0024980735765110723,
  (1, 1): 0.09847961692165914,
  (1, 2): 0.04483249953029945,
  (1, 3): 0.00011206739429775754,
  (1, 4): 0.0017172873392293946,
  (1, 6): 0.00324501813221815,
  (2, 1): 0.002874713386693711,
  (2, 2): 0.08341840343688685,
  (2, 4): 0.0003369771693204723,
  (2, 6): 0.004803531213092835,
  (3, 0): 0.016290621561706548,
  (3, 2): 0.12660296130200283,
  (3, 3): 0.0,
  (3, 4): 0.004962749405286196,
  (3, 6): 0.00045620245390119095,
  (4, 0): 0.02485535273939429,
  (4, 1): 0.0,
  (4, 2): 0.09921153588352019,
  (4, 3): 9.80817365512379e-05,
  (4, 4): 0.0004868469667218966,
  (4, 5): 0.0004801530427756057,
  (4, 6): 0.000918736169810105,
  (5, 0): 0.010521635444873225,
  (5, 1): 0.0017597428930459982,
  (5, 2): 0.051082454162700006,
  (5, 3): 0.0005626658738419525,
  (5, 4): 0.00581713906

### Compromise programming ###

In [14]:
from collections import Counter
from pymcdm import weights as obj_w
from compromise_programming import mcdm
from bayesian_model_evidence import log_evidence

assert len(pareto_optimal_models) >= 3
F = problem.evaluate(pareto_optimal_models)

ic = np.array([sm.OLS(y_pre, problem.numericalize_genome(bs[0])).fit().bic for bs in pareto_optimal_models])
max_ss = np.argmin(ic)+1
types = [-1, -1]

# ic = np.array([log_evidence(problem.numericalize_genome(bs[0]), y_pre) for bs in pareto_optimal_models])
# max_ss = np.argmax(ic)+1
# types = [+1, -1]

F = F[:max_ss]

obj_weights = obj_w.gini_weights(F, types=np.array([-1, -1]))
print("Weights:", obj_weights)

# mcdm
filtered_F = F.copy()
filtered_F[:, 0:1] = ic[:max_ss].reshape(-1, 1)
while len(filtered_F) >= 3:
    ranks, prefs = mcdm(filtered_F, obj_weights, types)
    most_common = Counter(np.argmin(ranks, axis=1)).most_common()
    filtered_F = F[:most_common[0][0]+1]
    print(filtered_F, most_common)
    if len(most_common) == 1:
        break

Weights: [0.35292656 0.64707344]
[[1.94434087e-05 1.00000000e+00]
 [6.72044820e-06 2.00000000e+00]] [(1, 3), (2, 1)]


In [19]:
X_pre_top = problem.numericalize_genome(top_candidates)

In [20]:
brute_force_all_subsets(X_pre, y_pre, max_support_size=2)

100%|██████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  8.29it/s]


(array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        , -0.3369438 ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.09297426,  0.        ,
          0.        ,  0.        ,  0.        , -0.9744152 ,  0.        ,
          0.        ,  0.        ,  0.        ,  0